# Data Foundation & Wrangling

In [24]:
import pandas as pd
import numpy as np

# Load the datasets
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
category_translation = pd.read_csv('data/product_category_name_translation.csv')

## 1. Schema Understanding & Merging

In [25]:
# Merge orders with order_items
df = pd.merge(orders, order_items, on='order_id', how='left')

# Merge with products
df = pd.merge(df, products, on='product_id', how='left')

# Merge with customers
df = pd.merge(df, customers, on='customer_id', how='left')

# Merge with sellers
df = pd.merge(df, sellers, on='seller_id', how='left')

# Merge with category_translation
df = pd.merge(df, category_translation, on='product_category_name', how='left')

# --- Advanced Payment Features ---
# 1. Create a pivot table for payment types
payment_details = order_payments.pivot_table(
    index='order_id', 
    columns='payment_type', 
    values='payment_value', 
    aggfunc='sum', 
    fill_value=0
).reset_index()

# 2. Get other aggregated payment features
payment_agg_features = order_payments.groupby('order_id').agg(
    payment_installments_max=('payment_installments', 'max'),
    payment_types_count=('payment_type', 'nunique')
).reset_index()

# 3. Merge payment features together
payment_final = pd.merge(payment_details, payment_agg_features, on='order_id', how='left')

# --- Advanced Review Features ---
# 1. Aggregate review scores to get mean, std, min, max
review_agg = order_reviews.groupby('order_id')['review_score'].agg(['mean', 'std', 'min', 'max']).reset_index()

# 2. Rename columns for clarity
review_agg.columns = ['order_id', 'review_score_mean', 'review_score_std', 'review_score_min', 'review_score_max']

# --- Merge all features back to the main dataframe ---
df = pd.merge(df, payment_final, on='order_id', how='left')
df = pd.merge(df, review_agg, on='order_id', how='left')

df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,credit_card,debit_card,not_defined,voucher,payment_installments_max,payment_types_count,review_score_mean,review_score_std,review_score_min,review_score_max
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,...,18.12,0.0,0.0,20.59,1.0,2.0,4.0,NaN,4.0,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,...,0.00,0.0,0.0,0.00,1.0,1.0,4.0,NaN,4.0,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,...,179.12,0.0,0.0,0.00,3.0,1.0,5.0,NaN,5.0,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,72.20,0.0,0.0,0.00,1.0,1.0,5.0,NaN,5.0,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,28.62,0.0,0.0,0.00,1.0,1.0,5.0,NaN,5.0,5.0


## 2. Cleaning & Feature Engineering

In [26]:
# Convert date columns to datetime objects
date_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'shipping_limit_date']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

# Calculate time differences
df['order_processing_time'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600 # in hours
df['shipping_time'] = (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / (3600 * 24) # in days
df['delivery_delay'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.total_seconds() / (3600 * 24) # in days

# Clean ZIP codes (example: ensure 5-digit format)
df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].astype(str).str.zfill(5)
df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].astype(str).str.zfill(5)

# --- Advanced Missing Value Handling ---
print("Missing value percentage in time-related columns before handling:")
print(df[['order_processing_time', 'shipping_time', 'delivery_delay']].isnull().mean() * 100)
df.dropna(subset=['order_processing_time', 'shipping_time', 'delivery_delay'], inplace=True)

# Impute product category name before using it for grouping
df['product_category_name'].fillna('unknown', inplace=True)

# Advanced imputation for product dimensions using category median
df['product_weight_g'] = df.groupby('product_category_name')['product_weight_g'].transform(lambda x: x.fillna(x.median()))
df['product_length_cm'] = df.groupby('product_category_name')['product_length_cm'].transform(lambda x: x.fillna(x.median()))
df['product_height_cm'] = df.groupby('product_category_name')['product_height_cm'].transform(lambda x: x.fillna(x.median()))
df['product_width_cm'] = df.groupby('product_category_name')['product_width_cm'].transform(lambda x: x.fillna(x.median()))

# --- Text Preprocessing ---
# Impute english category name and then standardize it
df['product_category_name_english'].fillna('unknown', inplace=True)
df['product_category_name_english'] = df['product_category_name_english'].str.lower().str.replace('_', ' ')

# --- Advanced Time Feature Engineering ---
df['purchase_day_of_week'] = df['order_purchase_timestamp'].dt.dayofweek
df['purchase_hour_of_day'] = df['order_purchase_timestamp'].dt.hour
df['purchase_month'] = df['order_purchase_timestamp'].dt.month
df['is_delayed'] = (df['delivery_delay'] > 0).astype(int)

# --- Geolocation Feature Engineering ---
# 1. Process geolocation data to get unique coordinates per zip code
geo_avg = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().reset_index()

# Ensure the merge keys have the same data type and format (zero-padded string)
geo_avg['geolocation_zip_code_prefix'] = geo_avg['geolocation_zip_code_prefix'].astype(str).str.zfill(5)

# 2. Merge to get customer and seller coordinates
df = pd.merge(df, geo_avg, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'}, inplace=True)
df.drop(columns='geolocation_zip_code_prefix', inplace=True)

df = pd.merge(df, geo_avg, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
df.rename(columns={'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'}, inplace=True)
df.drop(columns='geolocation_zip_code_prefix', inplace=True)

# 3. Calculate Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

df['distance_seller_customer'] = haversine_distance(df['seller_lat'], df['seller_lng'], df['customer_lat'], df['customer_lng'])

# Display head of the final dataframe
print("Displaying the first 5 rows of the final processed data:")
display(df.head())

# Save the final dataframe to a new CSV file
print("\nSaving the processed data to 'data/olist_processed_dataset.csv'...")
df.to_csv('data/olist_processed_dataset.csv', index=False)
print("Done.")

Missing value percentage in time-related columns before handling:
order_processing_time    0.141944
shipping_time            2.847697
delivery_delay           2.846815
dtype: float64
Displaying the first 5 rows of the final processed data:


/var/folders/df/lc6_wp2n4q92k7tv9ry8rw2m0000gn/T/ipykernel_43055/1480682313.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_category_name'].fillna('unknown', inplace=True)
/var/folders/df/lc6_wp2n4q92k7tv9ry8rw2m0000gn/T/ipykernel_43055/1480682313.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,delivery_delay,purchase_day_of_week,purchase_hour_of_day,purchase_month,is_delayed,customer_lat,customer_lng,seller_lat,seller_lng,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,...,-7.107488,0,10,10,0,-23.576983,-46.587161,NaN,NaN,NaN
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,...,-5.355729,1,20,7,0,-12.177924,-44.660711,NaN,NaN,NaN
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,...,-17.245498,2,8,8,0,-16.745150,-48.514783,NaN,NaN,NaN
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,-12.980069,5,19,11,0,-5.774190,-35.271143,NaN,NaN,NaN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,-9.238171,1,21,2,0,-23.676370,-46.514627,NaN,NaN,NaN



Saving the processed data to 'data/olist_processed_dataset.csv'...
Done.
